In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("transformations_actions")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/02 06:33:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 스파크 설정 환경 확인
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '41951'),
 ('spark.app.id', 'local-1685687618912'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'transformations_actions'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', 'ip-172-31-40-173.ap-northeast-3.compute.internal'),
 ('spark.app.startTime', '1685687617184'),
 ('spark.ui.showConsoleProgress', 'true')]

In [4]:
# pyhon 리스트를 RDD로 만들기
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [5]:
# RDD내의 모든 데이터 가져오기
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [6]:
# 각 데이터별 개수 세기
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [7]:
# 상위 n개의 데이터 가져오기 
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [8]:
# 처음 1개의 데이터만 가져오기
foods.first()

'짜장면'

In [9]:
# RDD내의 데이터 개수 세기
foods.count()

12

In [11]:
# 중복데이터 제거 (Transformations)
fd = foods.distinct()
fd

PythonRDD[9] at RDD at PythonRDD.scala:53

In [12]:
fd.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [13]:
# 중복을 제외한 개수는 distinct 후 count
foods.distinct().count()

7

요소를 하나씩 꺼내서 함수에 저장할 때 사용 : foreach

In [15]:
# Woker 에서 발생하는 일을 Master에서 print
foods.foreach(lambda x : print(x))

짜장면
마라탕
짬뽕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


# Narrow Transformations
- 1:1 변환을 의미
- 하나의 열을 조작하기 위해 다른 열 및 파티션의 데이터를 사용하지 않는다.
- filter(), map(), flatmap(), sample(), union()

In [16]:
sample_rdd = sc.parallelize([1, 2, 3])
sample_rdd

ParallelCollectionRDD[16] at readRDDFromFile at PythonRDD.scala:274

In [17]:
sample_rdd2 = sample_rdd.map(lambda x : x + 2)
sample_rdd2

PythonRDD[17] at RDD at PythonRDD.scala:53

In [18]:
sample_rdd2.collect()

[3, 4, 5]

In [19]:
# flatMap: 모든 결과를 1차원식으로 평탄화
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [20]:
moviesRDD = sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[18] at readRDDFromFile at PythonRDD.scala:274

In [25]:
mapmovies = moviesRDD.map(lambda x : x.split())

In [26]:
mapmovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [28]:
flatMovies = moviesRDD.flatMap(lambda x : x.split())

In [29]:
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

# Wide Transformations

In [30]:
# pyhon 리스트를 RDD로 만들기
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[21] at readRDDFromFile at PythonRDD.scala:274

grouBy(<func>)
   - <func> : 기준을 마련하는 함수

In [33]:
foodsGroup = foods.groupBy(lambda x : x[0])

In [32]:
res = foodsGroup.collect()
res

[('짜', <pyspark.resultiterable.ResultIterable at 0x7f03ac71bf10>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7f03ac71bdf0>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7f03ac71bbe0>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7f03ac71b340>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7f03ac71b250>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7f03ac71be50>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7f03ac71b3a0>)]

In [34]:
for (k, v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


In [35]:
sc.stop()